# Tensorflow Implementation

## Imports

We are importing all the neccessary libraries that we are gonna use.

In [26]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [27]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
# module_url = "../data"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [16]:
sentences = ["Python is a good language","Language a good python is"]

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_sentences_encodings = model(similarity_input_placeholder)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
  
    sentences_embeddings = session.run( similarity_sentences_encodings, feed_dict={
        similarity_input_placeholder: sentences
    })
  
    similarity = np.inner(sentences_embeddings[0], sentences_embeddings[1])
  
    print("Similarity is %s" % similarity)

Similarity is 0.92217684


In [4]:
df = pd.read_csv("../data/zeamed-web/zeamed-faq.csv")
df.head(5)

,Question,Answer
0,Does the Zeamed provide 24/7 customer support?,Yes
1,Is it a free service?,yes
2,Why is Zeamed providing a free service?,ZeaMed is absolutely free for consumers.Our a...
3,Where do I find zeamed doctor Profile on this ...,Select the test name and location and click on...
4,Will the patient info be secured?,Yes patient information/data is secured as per...


In [25]:
query = "who are you ?"

for question in df.Question:
    sentences = [ question , query ]

    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_sentences_encodings = model(similarity_input_placeholder)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
  
        sentences_embeddings = session.run( similarity_sentences_encodings, feed_dict={
            similarity_input_placeholder: sentences
        })
  
        similarity = np.inner(sentences_embeddings[0], sentences_embeddings[1])
  
        print("Similarity between %s & %s is %s" % (question , query , similarity) )

Similarity between Does the Zeamed provide 24/7 customer support? & who are you ? is -0.04937468
Similarity between Is it a free service? & who are you ? is 0.082572356
Similarity between Why is Zeamed providing a free service? & who are you ? is 0.035000674
Similarity between Where do I find zeamed doctor Profile on this app? & who are you ? is 0.089377984
Similarity between Will the patient info be secured?  & who are you ? is -0.072951436
Similarity between Who are ZeaMed’s users? & who are you ? is 0.5057231
Similarity between What is ZeaMed & who are you ? is 0.22169513
Similarity between If we book an appointment through you (Zeamed.com) do I stand a better chance of getting an appointment? & who are you ? is 0.04126934
Similarity between Are there any additional features I have access to with this app? & who are you ? is -0.013125197
Similarity between How do I install the zeamed app for doctors? & who are you ? is -0.036037218
Similarity between Is it free to use? & who are you

In [36]:
print(df.Question.shape)
shape = ( df.Question.shape[0] , 1 )
print(shape)

# questions_array = []
# for question in df.Question:
#    questions_array.append(question)
    
questions_array = np.array(df.Question).reshape(-1,1)

#print(questions_array)

#questions= tf.constant( questions_array , dtype=tf.string  ) 
query = tf.placeholder( tf.string , shape=(None) )
#embed_questions = model(questions)
embed_query = model(query)

with tf.Session() as session:
    session.run( tf.global_variables_initializer() )
    session.run( tf.tables_initializer() )
    
    #question_matrix = tf.run(embed_questions)
    query_matrix = session.run(embed_query , feed_dict = {
        query : "who are you"
    } )
    
    print( query_matrix)

(22,)
(22, 1)


InvalidArgumentError: [_Derived_] input must be a vector, got shape: []
	 [[{{node text_preprocessor/tokenize/StringSplit/StringSplit}}]]
	 [[StatefulPartitionedCall_20/StatefulPartitionedCall/StatefulPartitionedCall]]